In [1]:
from langchain.agents import Tool
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
#ChatGoogleGenerativeAI(google_api_key=api_key,model="gemini-1.5-flash-latest", temperature=0.3)
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

# Retrieve the API key
openai_api_key = os.getenv("OPENAI_API_KEY")
#api_key = os.getenv("google_api_key")

if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY not found in environment variables")
from pydantic.v1 import BaseModel, Field
class DocumentInput(BaseModel):
    question: str = Field()


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

tools = []
files = [
    # https://abc.xyz/investor/static/pdf/2023Q1_alphabet_earnings_release.pdf
    {
        "name": "Geeks_For_Geeks",
        "path": "GFG.txt",
    },
    # https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q1-2023-Update
    {
        "name": "CloudFare",
        "path": "cf.txt",
    },
    {
        "name": "ibm",
        "path": "ibm.txt",
    },
    
]

for file in files:
    loader = TextLoader(file["path"],encoding = 'UTF-8')
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(pages)
    embeddings = OpenAIEmbeddings()
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()

    # Wrap retrievers in a Tool
    tools.append(
        Tool(
            args_schema=DocumentInput,
            name=file["name"],
            description=f"useful when you want to answer questions about {file['name']}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
        )
    )

from langchain.agents import AgentType, initialize_agent
import os
llm = ChatOpenAI(
temperature=0.5,
model="gpt-3.5-turbo"
)
print(llm)
# llm=ChatGoogleGenerativeAI(google_api_key=api_key,model="gemini-1.5-flash-latest", temperature=0.3)

agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
    
)


k=agent({"input": "How Geeks for Geeks explained LLms as compare to ibm and cloudfare?"})
# prompt = hub.pull("hwchase17/react")
# agent = create_react_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# agent_executor.invoke({"input": "what is LangChain?"})

# prompt = hub.pull("hwchase17/react")
# agent = create_react_agent(llm, tools, prompt)
# agent_executor = AgentExecutor.from_agent_and_tools(agent = agent, tools=tools)
# #agent.invoke({"input": "How Geeks for Geeks explained LLms as compare to ibm and cloudfare?"})
# print(agent_executor)

print(k)

Created a chunk of size 2800, which is longer than the specified 1000
Created a chunk of size 1324, which is longer than the specified 1000
Created a chunk of size 1377, which is longer than the specified 1000
Created a chunk of size 1023, which is longer than the specified 1000


client=<openai.resources.chat.completions.Completions object at 0x0000016FF35A6B10> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000016FF35C0DD0> temperature=0.5 openai_api_key=SecretStr('**********') openai_proxy=''


> Entering new AgentExecutor chain...


c:\Users\deepa\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Users\deepa\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Invoking: `Geeks_For_Geeks` with `{'question': 'Explain LLMS'}`


{'query': 'Explain LLMS', 'result': 'Large Language Models (LLMs) are advanced artificial intelligence algorithms that utilize neural network techniques with a large number of parameters to process and understand human languages or text. These models are trained on extensive datasets using self-supervised learning techniques, where they learn intricate patterns and relationships from diverse language data during training. LLMs consist of multiple layers, including feedforward layers, embedding layers, and attention layers, and employ attention mechanisms like self-attention to capture dependencies and relationships in text sequences. LLMs can perform tasks such as text generation, machine translation, summary writing, image generation from texts, machine coding, chatbots, and conversational AI. They are highly efficient in capturing complex entity relationships in text and can generate text using the semantic and syntac

In [3]:
print(k['output'])

Geeks for Geeks explains Large Language Models (LLMs) as advanced artificial intelligence algorithms that utilize neural network techniques with a large number of parameters to process and understand human languages or text. These models are trained on extensive datasets using self-supervised learning techniques, where they learn intricate patterns and relationships from diverse language data during training. LLMs consist of multiple layers and employ attention mechanisms like self-attention to capture dependencies and relationships in text sequences. They can perform tasks such as text generation, machine translation, summary writing, image generation from texts, machine coding, chatbots, and conversational AI.

IBM describes LLMS as foundation models trained on immense amounts of data, enabling them to understand and generate natural language and other types of content for various tasks. They are popular for tasks like content generation, language translation, code writing, summariza

In [4]:
agent({"input": "Explain LLms of Cloudfare"})



> Entering new AgentExecutor chain...

Invoking: `CloudFare` with `{'question': 'Explain LLms of Cloudfare'}`


{'query': 'Explain LLms of Cloudfare', 'result': 'Cloudflare offers services that allow developers to quickly start building Large Language Models (LLMs). Developers can use services like Vectorize, a globally distributed vector database, to query data stored in object storage or documents stored in Workers Key Value. By combining this with Cloudflare Workers AI, developers can experiment with building their own LLM applications. LLMs are based on machine learning, specifically deep learning, to understand and generate text responses. They can be used for tasks like generating text in response to prompts or questions. However, there are security concerns related to LLMs, such as potential vulnerabilities to malicious inputs and the exposure of confidential data.'}Cloudflare offers services that allow developers to quickly start building Large Language Models (LLMs). Develop

{'input': 'Explain LLms of Cloudfare',
 'output': 'Cloudflare offers services that allow developers to quickly start building Large Language Models (LLMs). Developers can use services like Vectorize, a globally distributed vector database, to query data stored in object storage or documents stored in Workers Key Value. By combining this with Cloudflare Workers AI, developers can experiment with building their own LLM applications. LLMs are based on machine learning, specifically deep learning, to understand and generate text responses. They can be used for tasks like generating text in response to prompts or questions. However, there are security concerns related to LLMs, such as potential vulnerabilities to malicious inputs and the exposure of confidential data.'}